# Path of Exile Build Price Checker

### In this notebook, I will building a build price checker for the game Path of Exile. 

In [5]:
# imports
import pandas as pd
import pobapi
import requests
import re
import copy
import time
from bs4 import BeautifulSoup
import csv

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36"

In [ ]:
build_link = "https://pastebin.com/gSeiXiif"

In [6]:
# json import
# TODO change to reading from site https://www.pathofexile.com/api/trade/data/stats
mods = pd.read_json('poe_item_mods.json')
entries = mods['result'][1]['entries']
entry_dict = {}
for entry in entries: 
    entry_dict[entry['text']] = entry['id']

item_types = ['Claws', 'Daggers', 'Wands', 'One_Hand_Swords', 'Thrusting_One_Hand_Swords', 'One_Hand_Axes',
    'One_Hand_Maces', 'Sceptres', 'Rune_Daggers', 'Bows', 'Staves', 'Two_Hand_Swords', 'Two_Hand_Axes',
    'Two_Hand_Maces', 'Fishing_Rods', 'Quivers', 'Shields', 'Gloves', 'Boots', 'Body_Armours', 'Helmets',
    'Amulets', 'Rings', 'Belts', 'Jewels', 'Abyss_Jewels']

type_filter_dict = {
    'Claws': 'weapon.claw',
    'Daggers': 'weapon.dagger',
    'Wands': 'weapon.wand',
    'One_Hand_Swords': 'weapon.onesword',
    'One_Hand_Axe': 'weapon.oneaxe',
    'One_Hand_Mace': 'weapon.onemace',
    'Sceptres': 'weapon.sceptre',
    'Rune_Daggers': 'weapon.runedagger',
    'Bows': 'weapon.bow',
    'Staves': 'weapon.staff',
    'Warstaves': 'weapon.warstaff',
    'Two_Hand_Swords': 'weapon.twosword',
    'Two_Hand_Axes': 'weapon.twoaxe',
    'Two_Hand_Maces': 'weapon.twomace',
    'Fishing_Rods': 'weapon.rod',
    'Quivers': 'armour.quiver',
    'Shields': 'armour.shield',
    'Gloves': 'armour.gloves',
    'Boots': 'armour.boots',
    'Body_Armours': 'armour.chest',
    'Helmets': 'armour.helmet',
    'Amulets': 'accessory.amulet',
    'Rings': 'accessory.ring',
    'Belts': 'accessory.belt',
    'Jewels': 'jewel',
    'Abyss_Jewels': 'jewel.abyss'
}

# can scrape exact exchange rate later
chaosPerEx = 100

In [26]:
# scraping base types from poedb to match with item type (i.e. Hubris Circlet -> Helmet)
# NOTE: takes ~1.5-2 min to run

def item_scrape(item_types):
    headers = {"user-agent": USER_AGENT}
    item_dict = {}
    for item in item_types: 
        #print(item)
        url = f"http://www.poedb.tw/us/{item}#{item.replace('_', '')}Item"
        resp = requests.get(url, headers=headers).text
        soup = BeautifulSoup(resp, "html.parser")

        bases = get_base_names_implicit(soup, item)
        item_dict[item] = bases

        time.sleep(3)

        # Warstaffs special case
        if item == 'Staves': 
            url = f"http://www.poedb.tw/us/Staves#WarstavesItem"
            resp = requests.get(url, headers=headers).text
            soup = BeautifulSoup(resp, "html.parser")

            bases = get_base_names_implicit(soup, 'Warstaffs')
            item_dict[item] = bases
            time.sleep(3)

    return item_dict

item_dict = item_scrape(item_types)

def get_base_names(soup, item_type):
    typeItem = soup.find('div', attrs={'id':f'{item_type}Item'})
    if typeItem == None: 
        typeItem = soup.find('div', attrs={'id':f'Itemjson_item_classcn{item_type[:-1].replace("_", "")}'})
    tbody = typeItem.find('tbody')
    tds = tbody.findAll('td')
    bases = []
    for i in range(len(tds)):
        if i%2 == 1: 
            row = tds[i]
            name = row.find('a').text
            bases.append(name)
    return bases

def get_base_names_implicit(soup, item_type):
    typeItem = soup.find('div', attrs={'id':f'{item_type}Item'})
    if typeItem == None: 
        typeItem = soup.find('div', attrs={'id':f'Itemjson_item_classcn{item_type[:-1].replace("_", "")}'})
    tbody = typeItem.find('tbody')
    tds = tbody.findAll('td')
    bases = {}
    for i in range(len(tds)):
        if i%2 == 1: 
            row = tds[i]
            name = row.find('a').text
            implicit = row.find('span', attrs={'class': 'implicitMod'})
            if implicit: 
                implicit_value = implicit.text
            else: 
                implicit_value = implicit
            bases[name] = implicit_value
    return bases

def write_item_dict(item_dict):
    w = csv.writer(open("item_base_types.csv", "w"))
    for key, val in item_dict.items():
        w.writerow([key, val])

def read_item_dict(file_name):
    with open(file_name, mode='r') as inp:
        reader = csv.reader(inp)
        dict_from_csv = {rows[0]:rows[1] for rows in reader}
    return dict_from_csv

def base_type_mapper(item, item_dict, type_filter_dict = []): 
    for key, value in item_dict.items():
        if item in value.keys(): 
            if type_filter_dict: 
                return type_filter_dict[key]
            else: 
                return key
    return None

In [24]:
def resultParse(results): 
    string = ""
    for result in results: 
        string += result + ","
    return string[:-1]

def avgPriceFromListings(listings, chaosPerEx): 
    sumPrice = 0
    min_price = str(listings[0]['listing']['price']['amount']) + " " + str(listings[0]['listing']['price']['currency'])
    max_price = str(listings[-1]['listing']['price']['amount']) + " " + str(listings[-1]['listing']['price']['currency'])
    for listing in listings: 
        amount = listing['listing']['price']['amount']
        currency = listing['listing']['price']['currency']
        if currency == 'exalted': 
            sumPrice += amount * chaosPerEx
        elif currency == 'chaos': 
            sumPrice += amount
    return sumPrice / len(listings), min_price, max_price

def get_listings(query): 
    headers = {"user-agent": USER_AGENT}
    r = requests.post('https://www.pathofexile.com/api/trade/search/Expedition', json=query, headers=headers)
    #print(r.status_code, r.reason)
    result_id = r.json()['id']
    results = r.json()['result']

    if len(results) == 0: 
        #print('No results')
        return
    elif len(results) > 10: 
        result_string = resultParse(results[0:10])
    else: 
        result_string = resultParse(results)
    listings = requests.get(f"https://www.pathofexile.com/api/trade/fetch/{result_string}?query={result_id}", headers=headers)
    #print(listings.status_code, listings.reason)
    return listings.json()['result']

def getItems(pob_url):
    build = pobapi.from_url(pob_url)
    items = build.items
    non_unique = []
    unique = []
    for item in items: 
        if item.rarity == "Unique": 
            unique.append(item.name)
        else: 
            non_unique.append(item)
    return non_unique, unique

# Helper fns for item_to_query

def split_item_mods(item, item_dict):
    item_mods = item.text.split('\n')
    mod_edit = {}
    for mod in item_mods:
        value = re.findall(r'\d*\.?\d+', mod)
        mod_edit[(re.sub(r'(\+?\d*\.?\d+)', '#', mod))] = value

    # remove implicit mod(s)
    def remove_implicit(item, mod_edit, item_dict): 
        base = item.base
        implicit = ""
        for key, value in item_dict.items():
            if base in value.keys():
                implicit = value[base]
        mod_edit_new = {}
        if implicit: 
            implicit_edit = re.sub(r'\+?\(?\d*\–?\.?\d+\)?', '#', implicit)
            for key, value in mod_edit.items(): 
                if key != implicit_edit: 
                    mod_edit_new[key] = value
            return mod_edit_new
        else: 
            return mod_edit

    mod_edit_no_implicit = remove_implicit(item, mod_edit, item_dict)
    return mod_edit_no_implicit


def mod_text_to_id(mod_text, entry_dict):
    mod_with_id = copy.deepcopy(mod_text)
    for mod in mod_with_id: 
        if mod in entry_dict:
            mod_with_id[mod].append(entry_dict[mod])
    return mod_with_id

def gen_rare_mods(mods):
    filters = []
    for mod in mods: 
        mod_list = mods[mod]
        value = mod_list[0]
        mod_id = mod_list[1]
        filters.append({"id": f"{mod_id}", "value": {"min": value}},)
    return filters
    
def gen_rare_query(base_type, filters):
	rare_query = {
        "query": {
            "status": {
                "option": "online"
            },
            "stats": [{
                "type": "and",
                "filters": filters
            }],
            "filters": {
                "type_filters": {
                    "filters": {
                        "category": {
                            "option": base_type_mapper(base_type, item_dict, type_filter_dict)
                        }
                    }
                }
            }
        },
        "sort": {
            "price": "asc"
        }
    }
	return rare_query

def gen_rare_query_base(base_type, filters):
	rare_query = {
		"query": {
			"status": {
				"option": "online"
			},
			"type": base_type,
			"stats": [{
				"type": "and",
				"filters": filters
			}],
		},
		"sort": {
			"price": "asc"
		}
	}
	return rare_query

def rare_item_to_query(item, entry_dict, item_dict):
    base_type = item.base
    mod_edit = split_item_mods(item, item_dict)
    mods_with_id = mod_text_to_id(mod_edit, entry_dict)
    filters = gen_rare_mods(mods_with_id)
    return gen_rare_query_base(base_type, filters)

def unique_item_to_query(item_name):
    unique_query = {
		"query": {
			"status": {
				"option": "online"
			},
            "name": item_name,
		},
		"sort": {
			"price": "asc"
		}
	}
    return unique_query

def item_summary(item, listings, item_avg_price, min_price, max_price):
    if type(item) == pobapi.models.Item: 
        print(item.base + ": ")
    else: 
        print(item)
    print(str(len(listings)) + " listings found.")
    print("Min price: " + min_price + ", " +  "Max price: " + max_price)
    print("Avg price: " + str(item_avg_price) + " chaos (" + str(round(item_avg_price/chaosPerEx, 2)) + " exalted)") 

In [20]:
def build_price_prediction(build_link, chaosPerEx, entry_dict, item_dict):
    non_unique, unique = getItems(build_link)
    total_price = 0
    for item in non_unique: 
        if item.base == 'Medium Cluster Jewel' or item.base == 'Small Cluster Jewel' or item.base == 'Large Cluster Jewel': 
            time.sleep(3)
            continue
        query = rare_item_to_query(item, entry_dict, item_dict)
        listings = get_listings(query)
        if listings == None: 
            print("There are no listings for: " + item.base)
            continue
        item_avg_price, min_price, max_price = avgPriceFromListings(listings, chaosPerEx)
        total_price += item_avg_price
        item_summary(item, listings, item_avg_price, min_price, max_price)
        time.sleep(3)
    
    for item in unique:
        query = unique_item_to_query(item)
        listings = get_listings(query)
        item_avg_price, min_price, max_price = avgPriceFromListings(listings, chaosPerEx)
        total_price += item_avg_price
        item_summary(item, listings, item_avg_price, min_price, max_price)
        time.sleep(3)
    
    print("The total price of your build in chaos is: ")
    return total_price

In [25]:
build_price_prediction(build_link, chaosPerEx, entry_dict, item_dict)

Cobalt Jewel: 
10 listings found.
Min price: 20 chaos, Max price: 45 exalted
Avg price: 1982.0 chaos (19.82 exalted)
There are no listings for: Apothecary's Gloves
Vermillion Ring: 
2 listings found.
Min price: 3 exalted, Max price: 250 exalted
Avg price: 12650.0 chaos (126.5 exalted)
Pinnacle Tower Shield: 
3 listings found.
Min price: 1 exalted, Max price: 10 exalted
Avg price: 500.0 chaos (5.0 exalted)
Cold Iron Point
10 listings found.
Min price: 20 chaos, Max price: 30 chaos
Avg price: 22.9 chaos (0.23 exalted)
The Devouring Diadem
10 listings found.
Min price: 19 chaos, Max price: 25 chaos
Avg price: 21.2 chaos (0.21 exalted)
The Red Nightmare
5 listings found.
Min price: 3 exalted, Max price: 10 exalted
Avg price: 620.0 chaos (6.2 exalted)
Rumi's Concoction
10 listings found.
Min price: 1 chaos, Max price: 5 chaos
Avg price: 4.3 chaos (0.04 exalted)
The total price of your build in chaos is: 


15800.4